In [ ]:
import requests as r
from bs4 import BeautifulSoup

In [ ]:
resp = r.get('https://books.toscrape.com/')
soup = BeautifulSoup(resp.content)

In [ ]:
len(soup.find_all())

In [ ]:
book_tags = soup.find_all('article', attrs={'class': "product_pod"})

In [ ]:
len(book_tags)

In [ ]:
def extract_book_data(book_tag):
  title = book_tag.find('h3').find('a')['title']
  price = book_tag.find('p', attrs={"class": "price_color"}).get_text()
  rating = book_tag.find('p', attrs={"class": "star-rating"})['class'][1]
  return title, price, rating

In [ ]:
b = extract_book_data(book_tags[2])

In [ ]:
def clean_price(price):
  return float("".join([char for char in price if char.isdigit() or char == '.']))

In [ ]:
clean_price(b[1])

In [ ]:
import re

def clean_price(price):
  return float(re.sub("[^0-9.]", '', price))

clean_price(b[1])

In [ ]:
def convert_rating(rating):
  rating_map = {
    "One": 1,
    "Two": 2,
    "Three": 3,
    "Four": 4,
    "Five": 5,
  }

  return rating_map[rating]

In [ ]:
def extract_book_data(book_tag):
  title = book_tag.find('h3').find('a')['title']
  price = book_tag.find('p', attrs={"class": "price_color"}).get_text()
  rating = book_tag.find('p', attrs={"class": "star-rating"})['class'][1]
  return {
    "title": title,
    "price": clean_price(price),
    "rating": convert_rating(rating)
  }

In [ ]:
book_tags = soup.find_all('article', attrs={"class": "product_pod"})
book_data = [extract_book_data(book_tag)for book_tag in book_tags]
book_data

In [ ]:
from random import choice

In [ ]:
extract_book_data(choice(book_tags))

In [ ]:
sum([book['price'] for book in book_data]) / len(book_data)

In [ ]:
[book['title'] for book in book_data if book['price'] < 20]

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(book_data)

In [ ]:
df

In [ ]:
df.price.mean()

In [ ]:
df[df.price < 20].title

In [ ]:
df.to_csv('books.csv', index=False)

## Functional search parameters

In [ ]:
soup.find_all(id='messages')

In [ ]:
soup.find_all(attrs={"id": "messages"})

In [ ]:
len(soup.find_all(attrs={"id": lambda x: x is not None}))

In [ ]:
len(soup.find_all(lambda x: x.has_attr('id')))

In [ ]:
def fiction_category_achor(tag):
  return tag.name == 'a' and 'category' in tag['href'] and "Fiction" in tag.text

In [ ]:
len(soup.find_all(fiction_category_achor))

## Text search

In [ ]:
len(soup.find_all(text='Fiction'))

In [ ]:
import re
re.compile('Fiction', re.I)

In [ ]:
text_matches = soup.find_all(text=re.compile('Fiction', re.I))

In [ ]:
[text.strip() for text in text_matches]

In [ ]:
all_text = list(soup.stripped_strings)
[text for text in all_text if "fiction" in text.lower()]

## Searching by CSS

In [ ]:
book_tags = soup.find_all('article', attrs={"class": "product_pod"})

In [ ]:
titles = []
for tag in book_tags:
  title = tag.find('h3').find('a')['title']
  titles.append(title)

titles

In [ ]:
title_tags = soup.select('article.product_pod > h3 > a')
titles = [tag['title'] for tag in title_tags]
titles

In [ ]:
soup.select('[title*=Human]')

In [ ]:
len(soup.select('button.btn-primary[data-loading-text][class*=primary]'))